using kaggle dataset: https://www.kaggle.com/datasets/sanlian/app-store-reviews-for-a-mobile-app?resource=download

```
# This is formatted as code

> Add blockquote


```



In [ ]:
import pandas as pd
import numpy as np

dfnolabel = pd.read_csv('app_store_reviews.csv', sep=";")
dflabel = pd.read_csv('labeled_app_store_reviews.csv', sep=";")
dfnolabel.dropna(axis=1,how='all',inplace=True)
dflabel.dropna(axis=1,how='all',inplace=True)

dfnolabel.head()
dflabel.head()

,date,platform,country,review,star,user_id,issue_flag,likes_count,dislike_count,label
0,7.07.2023,iOS,Australia,Love the itinerary sharing feature. It makes p...,5,13c95471-be12-42f6-95d1-fa3dd104909b,No,1,1,Premium Features
1,12.08.2023,Android,India,"The premium price is a bit high, but the featu...",4,945725ef-7277-4884-b717-9259af1df8d2,No,2,2,Premium Features
2,12.09.2023,iOS,UK,I can't share my trip details via WhatsApp dir...,5,e3d956b8-6e00-4099-84ca-f51406e6b3f7,No,5,3,User Feedbacks & Recommendations
3,12.07.2023,Android,Brazil,"The price of premium is too high, could be bet...",3,1fa559f7-a707-4f23-97f9-1cb92a0ba85b,No,0,0,Premium Features
4,24.09.2023,iOS,India,Smooth booking experience.,3,67934645-a9bf-4283-b809-1baa3f067b19,No,2,5,General Topics


In [ ]:
!pip install nltk

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import string
from nltk.stem import WordNetLemmatizer


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
stopwords = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

def remove_stopwords(text):
    if isinstance(text, str):
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word not in stopwords]
        return ' '.join(filtered_text)
    else:
        return text
# clean data to reduce dimensionality and make it easier to encode words
def preprocesstext(text):
  if isinstance(text, str):
    text = text.lower()
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = remove_stopwords(text)
    text = lemmatizer.lemmatize(text)
    return text
  else:
    return text

example = "I thought this product was very smart, but also very dangerous."
example = preprocesstext(example)
print(example)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


thought product smart also dangerous


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_word_embeddings(review):
  inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True, max_length=512)
  outputs = model(**inputs)

  with torch.no_grad():
    outputs = model(**inputs)

  embeddings = outputs.last_hidden_state
  embeddings = embeddings.squeeze(0).numpy()

  return embeddings

pre_proccessed_text = dfnolabel['review'].apply(preprocesstext)
word_vectors = pre_proccessed_text.apply(get_word_embeddings)
for idx, vectors in word_vectors.items():
    print(f"Review {idx+1}:")
    print(vectors)
    print("\n")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Review 1:
[[-0.2735053  -0.2294893  -0.14663845 ... -0.22538817  0.12250631
   0.21149455]
 [ 0.6313644  -0.20425522  0.24692506 ...  0.279086    0.43017992
   0.13971247]
 [-0.6312029  -1.2581905   0.4329017  ...  0.33532667  0.1301811
   0.06995847]
 ...
 [-0.28227657 -0.5851398  -0.00140178 ... -0.2788684  -0.06101116
   0.386831  ]
 [-0.5852741  -0.43916613 -0.15498097 ...  0.24975167  0.6139535
  -0.7860772 ]
 [ 0.84951025  0.09115    -0.26674107 ...  0.34776455 -0.43683252
  -0.16084643]]


Review 2:
[[-0.5585228  -0.04916169  0.03215518 ... -0.377008   -0.01163022
   0.18116921]
 [-0.09003522 -0.80845845  0.5337765  ... -0.15301064  0.4688492
  -0.42310855]
 [-0.14869873 -1.0609092   0.5238063  ...  0.12881434 -0.18628813
  -0.5684202 ]
 ...
 [-0.33485824 -0.5148454   1.0965147  ... -0.68089646 -0.90154874
  -0.295482  ]
 [-0.5609844  -0.86079836 -0.18766692 ...  0.32014313 -0.20627655
  -0.33177906]
 [ 0.75365657  0.06807953 -0.39466116 ...  0.22809267 -0.6086855
  -0.14486971]

In [ ]:
#start training the data
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

svr = SVR()

rf_regressor = RandomForestRegressor(random_state=42)

linear_regressor = LinearRegression()
#

kernels = ['linear', 'poly', 'rbf', 'sigmoid']

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.01, 0.1, 1],
    'kernel': ['rbf', 'poly', 'sigmoid'] # Focus on potentially better kernels
}


features = word_vectors.values
labels = dfnolabel['star']

print(word_vectors.shape)
print(labels.shape)

def aggregate_embeddings(embeddings):
    return np.mean(embeddings, axis=0)  # Take the average of the word embeddings

# Apply to all reviews
aggregated_word_vectors = word_vectors.apply(aggregate_embeddings)

aggregated_word_vectors = np.vstack(aggregated_word_vectors.values)

grid_search = GridSearchCV(svr, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(aggregated_word_vectors, labels)

print("Best parameters found by Grid Search:")
print(grid_search.best_params_)
print("Best mean negative MSE score:")
print(grid_search.best_score_)
print("Best mean MSE score:")
print(-grid_search.best_score_)

# --- Evaluate Linear Regression ---
print("\nEvaluating Linear Regression:")

# Use cross_val_score with the Linear Regression model
# Scoring is still 'neg_mean_squared_error' for regression
linear_scores = cross_val_score(linear_regressor, aggregated_word_vectors, labels, cv=cv, scoring='neg_mean_squared_error')

# Convert negative MSE to positive MSE for easier interpretation
linear_mse_scores = -linear_scores

print(f"  Negative Mean Squared Error scores (Linear Regression): {linear_scores}")
print(f"  Mean Negative Mean Squared Error (Linear Regression): {np.mean(linear_scores):.3f}")
print(f"  Mean MSE (Linear Regression): {np.mean(linear_mse_scores):.3f}")
# --- End Linear Regression Evaluation ---
# for kernel in kernels:
#     model = SVR(kernel=kernel)
#     cv = KFold(n_splits=5, shuffle=True, random_state=42)
#     scores = cross_val_score(model, aggregated_word_vectors, labels, cv=cv, scoring='neg_mean_squared_error')
#     print(f"Kernel: {kernel}")
#     print(f"  Accuracy scores: {scores}")
#     print(f"  Mean accuracy: {np.mean(scores):.3f}")

features = word_vectors


# print("Accuracy: ", accuracy_score(labels, predictions))
# print(classification_report(labels, predictions))

final_model = SVR(kernel='rbf', C=10, gamma=0.01)
final_model.fit(aggregated_word_vectors, labels)
final_model.predict()


(321,)
(321,)
Best parameters found by Grid Search:
{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Best mean negative MSE score:
-1.2825208327694866
Best mean MSE score:
1.2825208327694866

Evaluating Linear Regression:
  Negative Mean Squared Error scores (Linear Regression): [-2.69342375 -2.25197697 -2.65278387 -2.12270927 -2.33129382]
  Mean Negative Mean Squared Error (Linear Regression): -2.410
  Mean MSE (Linear Regression): 2.410


SVR(C=10, gamma=0.01)